<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/AgenteDescuentosPerplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Agente Buscador de Descuentos

Este script implementa un chatbot en Gradio para buscar descuentos y reseñas de productos, específicamente orientado al mercado colombiano. Funciona en Google Colab y utiliza la API de Perplexity para procesar las consultas.

## Principales Características

- **Funcionalidades**: Busca descuentos/ofertas y reseñas de productos
- **Tecnologías**: Perplexity AI, Gradio, Google Colab
- **Herramientas implementadas**:
  - `buscar_descuentos`: Encuentra promociones actuales con enlaces directos
  - `buscar_resenas_producto`: Resume opiniones y valoraciones de productos

## Flujo de Trabajo

1. El usuario envía una consulta a través de la interfaz
2. El sistema analiza la intención mediante Perplexity AI
3. Basado en la intención detectada, ejecuta la herramienta apropiada
4. Presenta los resultados formateados en la interfaz de chat

## Configuración

- Requiere una API Key de Perplexity almacenada como secreto en Google Colab
- Los resultados incluyen enlaces clickeables a productos y ofertas
- La interfaz está construida con Gradio para una experiencia de usuario amigable

El chatbot está específicamente optimizado para buscar información relevante para el mercado colombiano en 2025.

---

Desarrollado por [Camilo Vega](https://www.linkedin.com/in/camilo-vega-169084b1/), Consultor IA.

In [ ]:
# Agente Buscador de Descuentos
# ==================================================================

# PASO 1: Instalación de dependencias
!pip install langchain langchain-core langchain-community openai python-dotenv requests gradio -q

# PASO 2: Importaciones
import os
import requests
import json
import re
from typing import Dict, List, Any, Optional
import traceback
import gradio as gr
from google.colab import userdata

# PASO 3: Configuración de API
API_URL_PERPLEXITY = "https://api.perplexity.ai/chat/completions"

# Obtener API Key de secretos de Colab
PERPLEXITY_API_KEY = None
try:
    PERPLEXITY_API_KEY = userdata.get('PERPLEXITY_KEY')
    if PERPLEXITY_API_KEY:
        print("✅ API Key de Perplexity cargada exitosamente desde los secretos de Colab.")
    else:
        print("⚠️ ADVERTENCIA: El secreto 'PERPLEXITY_KEY' existe pero podría estar vacío.")
except Exception as e:
    print(f"ℹ️ Información: No se pudo acceder a los secretos de Colab. Error: {e}")
    pass

if not PERPLEXITY_API_KEY:
    print("⛔ ERROR: API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Google Colab.")
    print("Para configurar un secreto en Colab:")
    print("1. Haz clic en el icono 🔑 en el panel lateral izquierdo")
    print("2. Agrega un nuevo secreto con nombre 'PERPLEXITY_KEY' y el valor de tu API key")

# PASO 4: Funciones para buscar descuentos y reseñas
# ==================================================================
def buscar_descuentos(query: str) -> str:
    """
    Busca descuentos y promociones actualizadas para productos o servicios.
    """
    print(f"🛠️ Herramienta buscar_descuentos activada con query: {query}")
    if not PERPLEXITY_API_KEY:
        return "Error: API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Colab."

    try:
        optimized_query = f"{query} descuentos ofertas promociones colombia 2025 actualizado"
        headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}", "Content-Type": "application/json"}

        # Sistema de prompt modificado para buscar descuentos e incluir enlaces
        system_prompt_tool = """
Eres un asistente experto en encontrar y formatear información sobre descuentos de productos.
Tu tarea es analizar la información disponible y presentarla de forma clara y ordenada.
Para cada descuento encontrado, debes generar una entrada con la siguiente estructura:

**Producto:** [Nombre del producto o descripción breve]
**Tienda/Fuente:** [Nombre de la tienda o fuente de la oferta, si se conoce]
**Descuento:** [Porcentaje o monto del descuento claramente indicado, ej. "48% de descuento", "Ahorro de $200.000"]
**Precio Original:** [Precio antes del descuento, si está disponible]
**Precio con Descuento:** [Precio después del descuento, si está disponible]
**Detalles Adicionales:** [Cualquier otra información relevante como garantía, envío, características destacadas, etc.]
**Enlace:** [URL completa y directa a la oferta, asegúrate de incluir el enlace real y completo]
--- (separador para la siguiente oferta)

IMPORTANTE: Siempre incluye una URL real para el "Enlace", incluso si debes buscarla o construirla de las fuentes que estás consultando. Este debe ser un enlace clickeable y válido a la tienda o sitio web donde se muestra la oferta.

Si encuentras múltiples ofertas, listalas una tras otra usando el formato anterior.
Si no encuentras descuentos específicos, indícalo claramente.
Céntrate en la información de descuentos para Colombia y el año 2025 si es posible, basado en tu conocimiento.
"""
        user_prompt_tool = f"""
Basado en tu conocimiento, encuentra y formatea los 3-5 mejores descuentos disponibles para: '{optimized_query}'.
Presenta cada descuento usando la estructura detallada en las instrucciones del sistema.
Si mencionas un precio, aclara si es el original o el de descuento.
RECUERDA: Es CRÍTICO que incluyas un enlace válido y clickeable para cada producto, utilizando URLs reales y completas de tiendas colombianas.
"""

        model_for_tool = "sonar"

        data = {
            "model": model_for_tool,
            "messages": [
                {"role": "system", "content": system_prompt_tool},
                {"role": "user", "content": user_prompt_tool}
            ],
            "temperature": 0.2,
            "max_tokens": 1500
        }

        print(f"API Call (dentro de buscar_descuentos) para: {optimized_query} con modelo {data['model']}")
        response = requests.post(API_URL_PERPLEXITY, headers=headers, json=data)
        response.raise_for_status()

        # Obtener la respuesta de Perplexity
        formatted_response = response.json()['choices'][0]['message']['content']

        # Asegurarnos de que los enlaces sean clickeables en Gradio
        formatted_response = convertir_enlaces_markdown(formatted_response)

        print(f"📄 Respuesta formateada de la herramienta buscar_descuentos:\n{formatted_response[:200]}...")
        return formatted_response

    except requests.exceptions.HTTPError as http_err:
        error_detail_msg = "No se pudo obtener detalle del error de la API."
        try:
            error_json = http_err.response.json()
            error_detail_msg = error_json.get("error", {}).get("message", http_err.response.text)
        except json.JSONDecodeError:
            error_detail_msg = http_err.response.text
        full_error_message = f"Error HTTP {http_err.response.status_code} al buscar descuentos: {error_detail_msg}"
        print(f"❌ {full_error_message}")
        traceback.print_exc()
        return full_error_message
    except Exception as e:
        print(f"❌ Error genérico en buscar_descuentos: {e}")
        traceback.print_exc()
        return f"Error al buscar descuentos: {str(e)}"

def buscar_resenas_producto(producto: str) -> str:
    """
    Busca reseñas y opiniones sobre un producto específico.
    """
    print(f"🛠️ Herramienta buscar_resenas_producto activada con producto: {producto}")
    if not PERPLEXITY_API_KEY:
        return "Error: API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Colab."

    try:
        optimized_query = f"{producto} review reseñas opiniones comparativa colombia"
        headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}", "Content-Type": "application/json"}

        system_prompt_tool = """Eres un experto analista de productos. Sintetiza reseñas de múltiples fuentes
        para dar un resumen equilibrado (pros, contras, calificación general si es posible).
        Usa información de Colombia si es relevante y disponible.
        Estructura tu respuesta claramente e incluye enlaces a tiendas donde se pueda comprar el producto."""

        user_prompt_tool = f"Busca y resume reseñas sobre: {optimized_query}. Incluye puntos fuertes, débiles, una conclusión y enlaces a tiendas donde se pueda adquirir el producto en Colombia."

        model_for_tool = "sonar-deep-research"

        data = {
            "model": model_for_tool,
            "messages": [
                {"role": "system", "content": system_prompt_tool},
                {"role": "user", "content": user_prompt_tool}
            ],
            "temperature": 0.3,
            "max_tokens": 1000
        }

        print(f"API Call (dentro de buscar_resenas_producto) para: {optimized_query} con modelo {data['model']}")
        response = requests.post(API_URL_PERPLEXITY, headers=headers, json=data)
        response.raise_for_status()

        result = response.json()['choices'][0]['message']['content']
        # Convertir enlaces a formato markdown clickeable
        result = convertir_enlaces_markdown(result)
        return result

    except requests.exceptions.HTTPError as http_err:
        error_detail_msg = "No se pudo obtener detalle del error de la API."
        try:
            error_json = http_err.response.json()
            error_detail_msg = error_json.get("error", {}).get("message", http_err.response.text)
        except json.JSONDecodeError:
            error_detail_msg = http_err.response.text
        full_error_message = f"Error HTTP {http_err.response.status_code} al buscar reseñas: {error_detail_msg}"
        print(f"❌ {full_error_message}")
        traceback.print_exc()
        return full_error_message
    except Exception as e:
        print(f"❌ Error genérico en buscar_resenas_producto: {e}")
        traceback.print_exc()
        return f"Error al buscar reseñas: {str(e)}"

# PASO 5: Funciones de utilidad para procesar respuestas
def convertir_enlaces_markdown(texto):
    """
    Convierte enlaces en el texto a formato markdown clickeable para Gradio
    También busca URLs no formateadas y las convierte en enlaces clickeables
    """
    # Asegúrate de que los enlaces en formato markdown se mantengan
    # Y busca URLs planas para convertirlas en enlaces
    url_pattern = r'https?://[^\s)"]+'

    # Primero, vamos a marcar los enlaces que ya están en formato markdown para no duplicarlos
    marked_texto = texto.replace('**Enlace:**', '**Enlace_MARKED:**')

    # Encontrar todas las URLs que no son parte de un enlace markdown
    urls = re.findall(url_pattern, marked_texto)
    for url in urls:
        # Verificar si esta URL ya es parte de un enlace markdown
        if f'[' not in url and '](' not in url and not url.endswith(')'):
            # Reemplazar la URL plana con un enlace markdown
            marked_texto = marked_texto.replace(url, f'[{url}]({url})')

    # Restaurar los marcadores originales
    return marked_texto.replace('**Enlace_MARKED:**', '**Enlace:**')

def decidir_accion_con_perplexity(user_input: str, current_chat_history: List[Dict[str, str]]) -> Dict[str, Any]:
    if not PERPLEXITY_API_KEY:
        return {"action": "error", "detail": "API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Colab."}

    system_message_content = """
Eres un asistente de chat que ayuda a los usuarios a encontrar descuentos y reseñas de productos.
Tienes disponibles las siguientes herramientas:
1. `buscar_descuentos`: Úsala si el usuario pide descuentos, ofertas, promociones. Necesita un `query` (string).
   Ejemplo de uso: si el usuario dice "descuentos en laptops", debes generar:
   {"action": "buscar_descuentos", "query": "laptops"}
2. `buscar_resenas_producto`: Úsala si el usuario pide opiniones o reseñas de un producto. Necesita un `producto` (string).
   Ejemplo de uso: si el usuario dice "qué tal es el iphone 15", debes generar:
   {"action": "buscar_resenas_producto", "producto": "iphone 15"}
Si la pregunta es un saludo, una conversación general, o no requiere una herramienta, responde directamente. En ese caso, genera:
{"action": "responder_directamente", "respuesta": "Tu respuesta aquí."}
Si no estás seguro o la pregunta es ambigua, pide clarificación generando:
{"action": "pedir_clarificacion", "respuesta": "Tu pregunta de clarificación aquí."}
Analiza la última entrada del usuario en el contexto del historial.
IMPORTANTE: Tu respuesta DEBE ser un objeto JSON válido con una clave "action" y otras claves dependiendo de la acción. No incluyas ningún otro texto fuera del JSON.
Acciones posibles para "action": "buscar_descuentos", "buscar_resenas_producto", "responder_directamente", "pedir_clarificacion".
"""
    messages_for_api = [{"role": "system", "content": system_message_content}]
    messages_for_api.extend(current_chat_history)
    messages_for_api.append({"role": "user", "content": user_input})

    headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": "sonar-pro",
        "messages": messages_for_api,
        "temperature": 0.1,
        "max_tokens": 200
    }

    print(f"🧠 Decidiendo acción para: '{user_input}' con el modelo {data['model']}")
    try:
        response = requests.post(API_URL_PERPLEXITY, headers=headers, json=data)
        response.raise_for_status()
        content_raw = response.json()['choices'][0]['message']['content']
        print(f"💡 Respuesta cruda de Perplexity (decisión): {content_raw}")

        content_cleaned = content_raw.strip()
        if content_cleaned.startswith("```json"):
            content_cleaned = content_cleaned[7:-3].strip()
        elif content_cleaned.startswith("```"):
            content_cleaned = content_cleaned[3:-3].strip()

        json_start_index = content_cleaned.find('{')
        json_end_index = content_cleaned.rfind('}')

        if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
            json_str = content_cleaned[json_start_index : json_end_index+1]
            print(f"🔧 JSON extraído para parsear: {json_str}")
            decision = json.loads(json_str)
        else:
            print(f"🔧 No se encontró un JSON claro, intentando parsear: {content_cleaned}")
            decision = json.loads(content_cleaned)

        if "action" not in decision:
            raise ValueError("La respuesta JSON de Perplexity no contiene la clave 'action'.")

        return decision

    except json.JSONDecodeError as je:
        print(f"❌ Error decodificando JSON de Perplexity: {je}. Contenido recibido: '{content_raw}'")
        return {"action": "error", "detail": f"Respuesta inesperada del asistente (no es JSON válido). Contenido: {content_raw}"}
    except requests.exceptions.HTTPError as http_err:
        error_msg_from_api = "N/A"
        texto_respuesta = http_err.response.text
        try:
            error_msg_from_api = http_err.response.json().get('error',{}).get('message', texto_respuesta[:200])
        except:
            error_msg_from_api = texto_respuesta[:200]
        print(f"❌ Error HTTP al decidir acción: {http_err}. Respuesta: {texto_respuesta}")
        return {"action": "error", "detail": f"Error de comunicación con el asistente: {http_err.response.status_code}, Mensaje: {error_msg_from_api}"}
    except Exception as e:
        print(f"❌ Error inesperado al decidir acción: {e}")
        traceback.print_exc()
        return {"action": "error", "detail": f"Error inesperado del asistente: {str(e)}"}

def procesar_consulta(mensaje_usuario: str, chat_history: List[tuple]) -> str:
    """
    Procesa la consulta del usuario y devuelve la respuesta para la interfaz de Gradio
    """
    print(f"▶️ Procesando consulta: '{mensaje_usuario}'")

    # Convertir historial de chat de Gradio a formato para el LLM
    chat_history_para_decision = []
    for human_msg, bot_msg in chat_history:
        if human_msg and bot_msg:  # Solo agregar mensajes completos
            chat_history_para_decision.append({"role": "user", "content": human_msg})
            chat_history_para_decision.append({"role": "assistant", "content": bot_msg})

    # Decidir la acción a realizar
    decision = decidir_accion_con_perplexity(mensaje_usuario, chat_history_para_decision)
    action = decision.get("action")

    # Ejecutar la acción correspondiente
    if action == "buscar_descuentos":
        query_param = decision.get("query")
        if query_param:
            print(f"➡️ Ejecutando herramienta 'buscar_descuentos' con query: '{query_param}'")
            respuesta = buscar_descuentos(query_param)
        else:
            respuesta = "El asistente decidió buscar descuentos pero no especificó qué buscar."
    elif action == "buscar_resenas_producto":
        producto_param = decision.get("producto")
        if producto_param:
            print(f"➡️ Ejecutando herramienta 'buscar_resenas_producto' con producto: '{producto_param}'")
            respuesta = buscar_resenas_producto(producto_param)
        else:
            respuesta = "El asistente decidió buscar reseñas pero no especificó el producto."
    elif action == "responder_directamente":
        respuesta = decision.get("respuesta", "El asistente no proporcionó una respuesta.")
    elif action == "pedir_clarificacion":
        respuesta = decision.get("respuesta", "¿Podrías ser más específico?")
    elif action == "error":
        respuesta = f"Error del asistente: {decision.get('detail', 'Error desconocido.')}"
    else:
        respuesta = f"Acción desconocida ({action}) o no implementada por el asistente."

    return respuesta

# PASO 6: Interfaz Gradio
def crear_interfaz_gradio():
    with gr.Blocks(title="Buscador de Descuentos", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🛍️ Chatbot Buscador de Descuentos y Reseñas")

        if not PERPLEXITY_API_KEY:
            gr.Markdown("""
            ## ⚠️ API Key no configurada

            Por favor, configura el secreto `PERPLEXITY_KEY` en Google Colab antes de usar esta aplicación:
            1. Haz clic en el icono 🔑 en el panel lateral izquierdo
            2. Agrega un nuevo secreto con nombre `PERPLEXITY_KEY` y el valor de tu API key
            3. Ejecuta nuevamente la celda de código
            """)
        else:
            gr.Markdown("Pregúntame sobre ofertas, promociones, descuentos o reseñas de productos.")

        # Componentes de chat
        chatbot = gr.Chatbot(
            label="Conversación",
            bubble_full_width=False,
            height=450,
            show_copy_button=True
        )
        msg = gr.Textbox(
            label="Tu mensaje",
            placeholder="Escribe tu consulta aquí...",
            lines=2
        )

        # Botones
        with gr.Row():
            send_btn = gr.Button("Enviar", variant="primary")
            clear_btn = gr.Button("Limpiar Chat")

        # Estado para el historial de chat
        chat_history = gr.State([])

        # Función para enviar mensaje
        def user_input(mensaje, historial):
            if not mensaje.strip():
                return historial, historial

            if not PERPLEXITY_API_KEY:
                error_msg = "Error: API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Google Colab."
                nuevo_historial = historial + [(mensaje, error_msg)]
                return nuevo_historial, nuevo_historial

            # Obtener respuesta
            respuesta = procesar_consulta(mensaje, historial)

            # Actualizar historial y mostrar
            nuevo_historial = historial + [(mensaje, respuesta)]
            return nuevo_historial, nuevo_historial

        # Función para limpiar chat
        def clear_chat_history():
            return []

        # Mensaje inicial de bienvenida
        def set_initial_message():
            if PERPLEXITY_API_KEY:
                mensaje_inicial = "¡Hola! Soy tu asistente para encontrar descuentos y reseñas. ¿En qué puedo ayudarte hoy?"
            else:
                mensaje_inicial = "⚠️ API Key de Perplexity no configurada. Por favor, configura el secreto 'PERPLEXITY_KEY' en Google Colab antes de usar esta aplicación."

            return [(None, mensaje_inicial)]

        # Vincular eventos a funciones
        send_btn.click(user_input, [msg, chat_history], [chatbot, chat_history])
        msg.submit(user_input, [msg, chat_history], [chatbot, chat_history])
        clear_btn.click(clear_chat_history, None, [chat_history])
        clear_btn.click(lambda: None, None, [chatbot], queue=False)

        # Cargar el mensaje inicial de bienvenida
        demo.load(set_initial_message, [], [chatbot])

    # Retornar la demo
    return demo

# PASO 7: Ejecución Principal
if __name__ == "__main__" and 'google.colab' in str(get_ipython()):
    # Comprobar si la API key está disponible
    if PERPLEXITY_API_KEY:
        print(f"ℹ️ Probando API Key de Perplexity (para llamadas directas): '{PERPLEXITY_API_KEY[:5]}...'")
        test_headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}", "Content-Type": "application/json"}
        valid_test_model = "sonar-pro"
        print(f"   Usando modelo de prueba: {valid_test_model}")
        test_data = {"model": valid_test_model, "messages": [{"role": "user", "content": "Hello!"}], "max_tokens": 5}
        try:
            test_response = requests.post(API_URL_PERPLEXITY, headers=test_headers, json=test_data)
            print(f"Respuesta de prueba de API: Código de estado {test_response.status_code}")
            if test_response.status_code == 200:
                print(f"✅ ¡Prueba de API Key de Perplexity ({valid_test_model}) EXITOSA!")
            else:
                print(f"❌ ¡Prueba de API Key de Perplexity ({valid_test_model}) FALLIDA! Código: {test_response.status_code}")
                print(f"   Contenido de la respuesta: {test_response.text}")
        except Exception as e_test:
            print(f"❌ Excepción durante la prueba de API Key: {e_test}")
    else:
        print("⛔ API Key de Perplexity no encontrada en los secretos de Colab.")
        print("Por favor, configura el secreto 'PERPLEXITY_KEY' en Google Colab antes de usar esta aplicación.")

    print("\n🚀 Iniciando la interfaz Gradio...")
    # Configuración específica para Colab
    app = crear_interfaz_gradio()
    # Lanzar con public=True para que sea accesible desde fuera de Colab
    app.launch(debug=True, share=True)